In [2]:
import os
import shutil
import random

# val data sampling
data_path = '/data/ILSVRC2012/val'

# data 저장 위치
dir_path = '/home/shkim/QT_DeiT_small/reproduce/sampled_imagenet_val'
os.makedirs(dir_path, exist_ok=True)

# 각 클래스별로 하나씩 뽑아서 만든 dataset 저장
one_sample_dir = os.path.join(dir_path, 'one_sample_per_class')
os.makedirs(one_sample_dir, exist_ok=True)

# 각 클래스별로 10개씩 뽑아서 만든 dataset 저장
ten_samples_dir = os.path.join(dir_path, 'ten_samples_per_class')
os.makedirs(ten_samples_dir, exist_ok=True)

# 각 클래스 디렉토리 탐색
for class_name in os.listdir(data_path):
    class_dir = os.path.join(data_path, class_name)
    if not os.path.isdir(class_dir):
        continue

    # 클래스별 파일 목록
    files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]

    # 하나씩 샘플링
    sampled_files = random.sample(files, min(len(files), 10))

    # 하나의 샘플만 복사
    one_sample_file = sampled_files[0]
    shutil.copy(os.path.join(class_dir, one_sample_file), os.path.join(one_sample_dir, one_sample_file))

    # 최대 10개의 샘플 복사
    for file_name in sampled_files:
        shutil.copy(os.path.join(class_dir, file_name), os.path.join(ten_samples_dir, file_name))

print(f"샘플링 완료. 결과는 {dir_path} 디렉토리에 저장되었습니다.")

샘플링 완료. 결과는 /home/shkim/QT_DeiT_small/reproduce/sampled_imagenet_val 디렉토리에 저장되었습니다.


In [24]:
import torch
import timm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import numpy as np
from torchvision import transforms
from PIL import Image

def load_vit_model(model_id, device):
    """Load a ViT model from Hugging Face."""
    model = timm.create_model(model_id, pretrained=True).to(device)
    return model

def load_image_dataset(dataset_path, device):
    """Load and preprocess images from the given dataset path."""
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    image_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]
    images = []
    for img_path in image_files:
        img = Image.open(img_path).convert('RGB')
        img = transform(img)
        images.append(img)

    return torch.stack(images).to(device)

def plot_tensor(tensor, layer_idx):
    """Plot a 3D graph of the tensor's absolute values.

    Args:
        tensor (torch.Tensor): Tensor of shape (tokens, channels).
        layer_idx (int): Index of the layer for labeling the plot.
    """
    tokens, channels = tensor.shape
    x, y = np.meshgrid(range(tokens), range(channels))
    z = np.abs(tensor).cpu().numpy()

    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(x, y, z.T, cmap='viridis')

    ax.set_title(f'Layer {layer_idx} Output (|Values|)')
    ax.set_xlabel('Token Index')
    ax.set_ylabel('Channel')
    ax.set_zlabel('Value')
    plt.show()

def main(model_id, dataset_path):
    """Main function to load the model, dataset, and plot layer-wise tensor values."""
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model
    model = load_vit_model(model_id, device)
    model.eval()

    # Load dataset
    dataset = load_image_dataset(dataset_path, device)

    # Pass one sample through the model
    sample = dataset[0].unsqueeze(0)  # Use only one sample

    # Hook to extract layer outputs
    def hook_fn(module, input, output):
        layer_outputs.append(output)

    layer_outputs = []
    hooks = []

    # Register hooks on transformer layers
    for name, module in model.named_modules():
        if 'blocks' in name and 'mlp' not in name:  # Target only transformer blocks
            hooks.append(module.register_forward_hook(hook_fn))

    # Run inference
    with torch.no_grad():
        _ = model(sample)

    # Remove hooks
    for hook in hooks:
        hook.remove()

    # Plot each layer's output
    for idx, layer_output in enumerate(layer_outputs):
        # Assuming layer_output is of shape (1, tokens, channels)
        plot_tensor(layer_output.squeeze(0), idx + 1)

if __name__ == "__main__":
    MODEL_ID = 'hf_hub:timm/vit_large_patch14_dinov2.lvd142m'
    DATASET_PATH = "/home/shkim/QT_DeiT_small/reproduce/sampled_imagenet_val/one_sample_per_class"

    main(MODEL_ID, DATASET_PATH)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/shkim/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3348347/2401854689.py", line 96, in <module>
    main(MODEL_ID, DATASET_PATH)
  File "/tmp/ipykernel_3348347/2401854689.py", line 58, in main
    model = load_vit_model(model_id, device)
  File "/tmp/ipykernel_3348347/2401854689.py", line 12, in load_vit_model
    model = timm.create_model(model_id, pretrained=True).to(device)
  File "/home/shkim/anaconda3/envs/yom/lib/python3.8/site-packages/timm/models/factory.py", line 72, in create_model
  File "/home/shkim/anaconda3/envs/yom/lib/python3.8/site-packages/timm/models/hub.py", line 84, in load_model_config_from_hf
  File "/home/shkim/anaconda3/envs/yom/lib/python3.8/site-packages/timm/models/hub.py", line 58, in has_hf_hub
RuntimeError: Hugging Face hub model specified but package not installed. Run `pip install h

In [23]:
!pip install --upgrade timm huggingface_hub


  Using cached huggingface_hub-0.27.1-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.27.1-py3-none-any.whl (450 kB)
